In [66]:
import pandas as pd
import numpy as np

In [67]:
dfe_2023=pd.read_csv("../input/db/sales/electro-2023.csv", parse_dates=["Date"])
dfe_2024=pd.read_csv("../input/db/sales/electro-2024.csv",parse_dates=["Date"])
dfe_2025=pd.read_csv("../input/db/sales/electro-2025.csv",parse_dates=["Date"])
locations=pd.read_csv("../output/locations_cleaned.csv")
products=pd.read_csv("../output/Productos.csv")

In [68]:
dfe_2023.head()

,Date,DocumentNumber,Type,LocationId,ProductId,Quantity,TotalCost,TotalPrice,TotalDiscount,LocationStockId
0,2023-02-07,3200000001411001,Sale,1569,A175047091035018,1,641.12620,895.51072,99.500856,1512
1,2023-02-04,3200000001409001,Sale,1569,A173051025133010,1,57.30400,100.97568,0.000000,1512
2,2023-02-01,3200000001393001,Sale,1569,D175094032032010,2,658.51344,1330.53856,147.838288,1512
3,2023-01-27,3200000001372001,Sale,1569,A178045077020010,1,294.49732,469.90788,0.000000,1512
4,2023-01-24,3200000001347001,Sale,1569,V170073032033010,2,273.27976,508.95000,0.000000,1512


In [69]:
dfe_2023[dfe_2023["Type"]!="Sale"]

,Date,DocumentNumber,Type,LocationId,ProductId,Quantity,TotalCost,TotalPrice,TotalDiscount,LocationStockId
50235,2023-05-22,4103000000070101,Return,1644,D174051047053010,-1,-25.99792,-45.78288,0.000000,1644
50236,2023-05-29,4103000000074101,Return,1644,D178037025061010,-1,-10.25440,-26.91780,0.000000,1644
50269,2023-02-13,4103000000055101,Return,1644,A177055021082017,-1,-27.70196,-39.97708,-4.443216,1519
62549,2023-02-26,4103000000059101,Return,1644,B175057038050010,-1,-167.73484,-241.14428,-23.926018,1569
214801,2023-12-22,4102000002028008,Return,1514,V170078037024010,-1,-113.10000,-134.63424,-92.917049,1514
...,...,...,...,...,...,...,...,...,...,...
234506,2023-08-21,4142000000680010,Return,1566,O172043027081010,-1,-441.16540,-631.47500,0.000000,1640
234507,2023-09-14,4142000000806010,Return,1513,D171036045035010,-1,-333.46404,-483.37432,-188.500000,1513
234508,2023-09-25,4142000000865010,Return,1513,A179047065024010,-1,-266.91600,-422.93368,-46.990396,1640
234509,2023-10-23,4142000000960010,Return,1538,A175047072063010,-1,-180.96000,-262.07532,-46.263299,1640


In [70]:
#Unimos los dataferames de ventas de los tres años
electronics= pd.concat([dfe_2023, dfe_2024, dfe_2025], ignore_index=True)
electronics=electronics[electronics["Type"]=="Sale"]

In [71]:
electronics["venta"]=electronics["TotalPrice"]-electronics["TotalDiscount"]
electronics["ganancia"]=electronics["TotalPrice"]-electronics["venta"]

In [72]:
electronics['Category'] = electronics['ProductId'].map(products.set_index('idProd')['PCategory'])
electronics['SubCategory'] = electronics['ProductId'].map(products.set_index('idProd')['subCat'])
electronics['TipoTienda'] = electronics['LocationStockId'].map(locations.set_index('id')['type'])
electronics['DimensionesTienda'] = electronics['LocationStockId'].map(locations.set_index('id')['tamanio'])
electronics['Ciudad'] = electronics['LocationStockId'].map(locations.set_index('id')['city'])

In [73]:
#Estructurar el dataset final
df_agg = electronics.groupby(['Date', 'LocationId', 'LocationStockId','Category', 'SubCategory', 'TipoTienda',
       'DimensionesTienda', 'Ciudad'], as_index=False).agg({
    'venta': 'sum',
    'Quantity': 'sum'
})
df_agg.head()

,Date,LocationId,LocationStockId,Category,SubCategory,TipoTienda,DimensionesTienda,Ciudad,venta,Quantity
0,2023-01-01,1546,1546,ACCESORIOS,AUDIFONOS,TIENDA,MEDIANO,LA LIBERTAD,69.986280,2
1,2023-01-01,1546,1546,COCINA,SANDUCHERAS Y WAFLERAS,TIENDA,MEDIANO,LA LIBERTAD,39.042120,1
2,2023-01-01,1546,1546,PILAS,ALCALINA - AA,TIENDA,MEDIANO,LA LIBERTAD,12.109240,1
3,2023-01-01,1546,1546,VENTILADORES,MESA,TIENDA,MEDIANO,LA LIBERTAD,64.591953,2
4,2023-01-01,1546,1546,VENTILADORES,PEDESTAL,TIENDA,MEDIANO,LA LIBERTAD,45.767800,1


In [74]:
#feature Engineering luego de la union de los dataframes
festivos = pd.to_datetime(['2023-01-02', '2023-02-20', '2023-02-21', '2023-04-07','2023-05-01','2023-05-26','2023-08-11','2023-10-09','2023-11-02','2023-11-03','2023-12-25'
                           , '2024-01-01', '2024-02-12', '2024-02-23', '2024-03-29','2024-05-03','2024-05-24','2024-08-09','2024-10-11','2024-11-01','2024-11-04','2024-12-25',
                           '2025-01-01', '2025-03-03', '2025-03-04', '2025-04-18','2025-05-02','2025-05-23','2025-08-11','2025-10-10','2025-11-03','2025-11-04','2025-12-25'])
df_agg['es_festivo'] = df_agg['Date'].isin(festivos).astype(int)
df_agg['dia_semana'] = df_agg['Date'].dt.dayofweek
df_agg['mes'] = df_agg['Date'].dt.month
# Codificación cíclica semanal
df_agg['sin_semana'] = np.sin(2 * np.pi * df_agg['dia_semana'] / 7)
df_agg['cos_semana'] = np.cos(2 * np.pi * df_agg['dia_semana'] / 7)

# Codificación cíclica mensual
df_agg['sin_mes'] = np.sin(2 * np.pi * df_agg['mes'] / 12)
df_agg['cos_mes'] = np.cos(2 * np.pi * df_agg['mes'] / 12)

# Rezagos
df_agg['ventas_lag1'] = df_agg['venta'].shift(1)
df_agg['ventas_lag7'] = df_agg['venta'].shift(7)
df_agg['ventas_rolling7'] = df_agg['venta'].shift(1).rolling(window=7).mean()

In [75]:
df_agg = df_agg.dropna().reset_index(drop=True)

In [76]:
df_agg.head()

,Date,LocationId,LocationStockId,Category,SubCategory,TipoTienda,DimensionesTienda,Ciudad,venta,Quantity,es_festivo,dia_semana,mes,sin_semana,cos_semana,sin_mes,cos_mes,ventas_lag1,ventas_lag7,ventas_rolling7
0,2023-01-01,1553,1519,ACONDICIONADORES DE AIRE,AA 24.000 BTU,BODEGA,PEQUEÑO,GUAYAQUIL,907.488553,1,0,6,1,-0.781831,0.62349,0.5,0.866025,87.509240,69.986280,153.094930
1,2023-01-01,1553,1553,ACCESORIOS,AUDIFONOS,TIENDA,GRANDE,BABAHOYO,53.850680,1,0,6,1,-0.781831,0.62349,0.5,0.866025,907.488553,39.042120,272.738112
2,2023-01-01,1553,1553,ACCESORIOS,PARLANTE PORTATIL,TIENDA,GRANDE,BABAHOYO,107.703516,1,0,6,1,-0.781831,0.62349,0.5,0.866025,53.850680,12.109240,274.853621
3,2023-01-01,1553,1553,ACCESORIOS,SOPORTES TV,TIENDA,GRANDE,BABAHOYO,20.177040,1,0,6,1,-0.781831,0.62349,0.5,0.866025,107.703516,64.591953,288.509946
4,2023-01-01,1553,1553,COCINA,LICUADORAS,TIENDA,GRANDE,BABAHOYO,53.850680,1,0,6,1,-0.781831,0.62349,0.5,0.866025,20.177040,45.767800,282.164958


In [77]:
df_agg.dropna(inplace=True)
df_agg[df_agg.isna().all(axis=1)]

,Date,LocationId,LocationStockId,Category,SubCategory,TipoTienda,DimensionesTienda,Ciudad,venta,Quantity,es_festivo,dia_semana,mes,sin_semana,cos_semana,sin_mes,cos_mes,ventas_lag1,ventas_lag7,ventas_rolling7


In [78]:
df_agg.to_csv("../output/ModelDataset.csv", index=False)